# Overview

Notebook to explore the TCGA metadata datasets and clean some of them for accurate upload in BigQuery.

# Imports

In [3]:
import sys
#add the data path
sys.path.append('../data')

In [79]:
import numpy as np
import pandas as pd
from pathlib import Path

# File paths

In [20]:
data_dir = Path('../data/')

In [21]:
samples_path = Path(data_dir / 'sample.tsv')
slides_path = Path(data_dir / 'slide.tsv')

# EDA and cleanup of samples metadata

In [23]:
sample_df = pd.read_csv(samples_path, sep='\t')

In [30]:
sample_df.head()

,project.project_id,cases.case_id,cases.submitter_id,samples.biospecimen_anatomic_site,samples.biospecimen_laterality,samples.catalog_reference,samples.composition,samples.current_weight,samples.days_to_collection,samples.days_to_sample_procurement,...,samples.specimen_type,samples.state,samples.submitter_id,samples.time_between_clamping_and_freezing,samples.time_between_excision_and_freezing,samples.tissue_collection_type,samples.tissue_type,samples.tumor_code,samples.tumor_code_id,samples.tumor_descriptor
0,TARGET-AML,69b4ad12-4436-53bf-a552-cee9a1ae4025,TARGET-20-PATETC,'--,'--,'--,'--,'--,'--,'--,...,Bone Marrow NOS,released,TARGET-20-PATETC-09A,'--,'--,'--,Tumor,Acute myeloid leukemia (AML),20,Primary
1,BEATAML1.0-COHORT,4bcb5c53-d625-43ac-9448-ebe07b9d705f,2829,'--,'--,'--,'--,'--,'--,'--,...,Solid Tissue,released,BA3442D,'--,'--,'--,Normal,'--,'--,Not Applicable
2,BEATAML1.0-COHORT,4bcb5c53-d625-43ac-9448-ebe07b9d705f,2829,'--,'--,'--,'--,'--,'--,'--,...,Peripheral Blood NOS,released,BA3245R,'--,'--,'--,Tumor,'--,'--,Recurrence
3,BEATAML1.0-COHORT,4bcb5c53-d625-43ac-9448-ebe07b9d705f,2829,'--,'--,'--,'--,'--,'--,'--,...,Peripheral Blood NOS,released,BA3245D,'--,'--,'--,Tumor,'--,'--,Recurrence
4,MP2PRT-ALL,508996da-a6e0-4451-ac47-7f927d90c809,MP2PRT-PATIRV,Bone Marrow,'--,'--,Whole Bone Marrow,'--,'--,'--,...,Whole Bone Marrow,released,MP2PRT-PATIRV-TMP1-A,'--,'--,'--,Tumor,'--,'--,Primary


**There are many columns with majority values being '--**

In [51]:
print("Columns with a value '-- :'")
for col in sample_df.columns:
    if "'--" in sample_df[col].values:
        print(col)    

Columns with a value '-- :'
samples.biospecimen_anatomic_site
samples.biospecimen_laterality
samples.catalog_reference
samples.composition
samples.current_weight
samples.days_to_collection
samples.days_to_sample_procurement
samples.diagnosis_pathologically_confirmed
samples.distance_normal_to_tumor
samples.distributor_reference
samples.freezing_method
samples.growth_rate
samples.initial_weight
samples.intermediate_dimension
samples.is_ffpe
samples.longest_dimension
samples.method_of_sample_procurement
samples.oct_embedded
samples.passage_count
samples.pathology_report_uuid
samples.sample_ordinal
samples.sample_type
samples.sample_type_id
samples.shortest_dimension
samples.time_between_clamping_and_freezing
samples.time_between_excision_and_freezing
samples.tissue_collection_type
samples.tumor_code
samples.tumor_code_id


**Clean up those columns by replacing those values with None or np.nan based on the data type.**

In [52]:
# create a copy of the data
sample_df2 = sample_df.copy()

In [62]:
cols_to_clean = []

for col in sample_df2.columns:

    # values with string only values
    if "'--" in sample_df2[col].values:
        cols_to_clean.append(col)

    # now replace those values everywhere
    sample_df2[cols_to_clean] = sample_df2[cols_to_clean].replace("'--'", np.nan)

# check again

        
        
        
            

In [83]:
sample_df2[['samples.freezing_method', 'samples.oct_embedded']].eq("'--'").any(axis=1)

0        False
1        False
2        False
3        False
4        False
         ...  
88294    False
88295    False
88296    False
88297    False
88298    False
Length: 88299, dtype: bool

**So majority columns have uniform, string data type and can be cleaned up. However 2 columns need further inspections.**

In [63]:
problematic_cols

['samples.freezing_method', 'samples.oct_embedded']

* No numeric data type columns that also have '-- values

In [64]:
cols_to_clean_numeric

[]

In [65]:
cols_to_clean_string

['samples.biospecimen_anatomic_site',
 'samples.biospecimen_laterality',
 'samples.catalog_reference',
 'samples.composition',
 'samples.current_weight',
 'samples.days_to_collection',
 'samples.days_to_sample_procurement',
 'samples.diagnosis_pathologically_confirmed',
 'samples.distance_normal_to_tumor',
 'samples.distributor_reference',
 'samples.growth_rate',
 'samples.initial_weight',
 'samples.intermediate_dimension',
 'samples.is_ffpe',
 'samples.longest_dimension',
 'samples.method_of_sample_procurement',
 'samples.passage_count',
 'samples.pathology_report_uuid',
 'samples.sample_ordinal',
 'samples.sample_type',
 'samples.sample_type_id',
 'samples.shortest_dimension',
 'samples.time_between_clamping_and_freezing',
 'samples.time_between_excision_and_freezing',
 'samples.tissue_collection_type',
 'samples.tumor_code',
 'samples.tumor_code_id']

* Inspect the problematic ones.

In [78]:
sample_df2['samples.freezing_method'].value_counts(dropna=False)

samples.freezing_method
'--    73373
NaN     9483
LN2     3559
-20     1783
-80      101
Name: count, dtype: int64

In [77]:
sample_df2['samples.freezing_method'].apply(type).value_counts().index

Index([<class 'str'>, <class 'float'>], dtype='object', name='samples.freezing_method')

In [74]:
sample_df2.iloc[0]

project.project_id                                                            TARGET-AML
cases.case_id                                       69b4ad12-4436-53bf-a552-cee9a1ae4025
cases.submitter_id                                                      TARGET-20-PATETC
samples.biospecimen_anatomic_site                                                    '--
samples.biospecimen_laterality                                                       '--
samples.catalog_reference                                                            '--
samples.composition                                                                  '--
samples.current_weight                                                               '--
samples.days_to_collection                                                           '--
samples.days_to_sample_procurement                                                   '--
samples.diagnosis_pathologically_confirmed                                           '--
samples.distance_norm

In [43]:
sample_df['samples.biospecimen_anatomic_site'].value_counts()

samples.biospecimen_anatomic_site
'--                      78713
Blood                     3000
Bone Marrow               1512
Lung                       932
Kidney                     622
                         ...  
Large Bowel                  1
Chin                         1
Testis                       1
Lymph Node(s) Splenic        1
Appendix                     1
Name: count, Length: 106, dtype: int64